In [68]:
from pathlib import Path
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np

***XRIO

In [119]:
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [[-61.76745396167119,-35.54701215790192],
        [-57.19714146167119,-35.54701215790192],
        [-57.19714146167119,-33.2630710872506],
        [-61.76745396167119,-35.54701215790192],]
    ],
}

time_of_interest = "2022-01-01/2023-03-30" #NOTE: L8 launched 02/2013, L9 launched 09/2021

In [116]:
area_of_interest = [-60.00, -36.84, -58.00, -34.50]

In [110]:
print(bbox_of_interest)

[-60.0, -36.84, -58.0, -34.5]


In [117]:
import xarray as xr
import geopandas as gpd
import os

import planetary_computer 
import pystac_client
import odc.stac

import matplotlib.pyplot as plt

In [118]:
%%time

# if you have an account (https://planetarycomputer.developer.azure-api.net/profile)
#os.environ['PC_SDK_SUBSCRIPTION_KEY'] = 'fgh1cc0c158a493f9bce23564c667856'  
# Always set this when reading from URLs
os.environ['GDAL_DISABLE_READDIR_ON_OPEN']='EMPTY_DIR'

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

search = catalog.search(
    collections=["sentinel-2-l2a"],
    intersects= area_of_interest,
    datetime=time_of_interest,
    query={"eo:cloud_cover": {"lt": 10}},
)


items = search.get_all_items()
print(f"Returned {len(items)} Items")

Exception: intersects must be of type None, str, dict, or an object that implements __geo_interface__

In [89]:
# Items is a list of GeoJON Features, you can turn it into a GeoPandas table for convenient exploration:
gf = gpd.GeoDataFrame.from_features(items.to_dict(), crs='EPSG:4326')

gf.head(3)

,geometry,datetime,platform,proj:epsg,instruments,s2:mgrs_tile,constellation,s2:granule_id,eo:cloud_cover,s2:datatake_id,...,s2:cloud_shadow_percentage,s2:nodata_pixel_percentage,s2:unclassified_percentage,s2:dark_features_percentage,s2:not_vegetated_percentage,s2:degraded_msi_data_percentage,s2:high_proba_clouds_percentage,s2:reflectance_conversion_factor,s2:medium_proba_clouds_percentage,s2:saturated_defective_pixel_percentage
0,"POLYGON ((-60.29520 -35.19821, -59.09015 -35.2...",2023-03-29T13:47:09.024000Z,Sentinel-2B,32721,[msi],21HTA,Sentinel 2,S2B_OPER_MSI_L2A_TL_MSFT_20230329T214329_A0316...,4.454349,GS2B_20230329T134709_031653_N05.09,...,4.879802,0.000013,0.312079,0.000000,27.658752,0.0107,1.503456,1.005899,1.890754,0.0
1,"POLYGON ((-59.25299 -32.51632, -59.28378 -32.6...",2023-03-27T13:57:01.024000Z,Sentinel-2A,32721,[msi],21HTD,Sentinel 2,S2A_OPER_MSI_L2A_TL_MSFT_20230327T232850_A0405...,0.005178,GS2A_20230327T135701_040533_N05.09,...,0.000000,29.510382,1.511557,0.014379,21.916012,0.0307,0.000014,1.007036,0.000061,0.0
2,"POLYGON ((-59.49005 -33.41271, -59.51840 -33.5...",2023-03-27T13:57:01.024000Z,Sentinel-2A,32721,[msi],21HTC,Sentinel 2,S2A_OPER_MSI_L2A_TL_MSFT_20230327T232831_A0405...,0.003199,GS2A_20230327T135701_040533_N05.09,...,0.000000,47.621429,0.001013,0.000038,38.463461,0.0218,0.000000,1.007036,0.000025,0.0


In [90]:
gf.iloc[0]

geometry                                   POLYGON ((-60.295197 -35.1982077, -59.09015 -3...
datetime                                                         2023-03-29T13:47:09.024000Z
platform                                                                         Sentinel-2B
proj:epsg                                                                              32721
instruments                                                                            [msi]
s2:mgrs_tile                                                                           21HTA
constellation                                                                     Sentinel 2
s2:granule_id                              S2B_OPER_MSI_L2A_TL_MSFT_20230329T214329_A0316...
eo:cloud_cover                                                                      4.454349
s2:datatake_id                                            GS2B_20230329T134709_031653_N05.09
s2:product_uri                             S2B_MSIL2A_20230329T134709_

In [91]:
gf.explore(style_kwds=dict(fill=False), column='platform')

In [92]:
# The dtypes of data often need adjusting!
gf.datetime.dtype

dtype('O')

In [93]:
gf['datetime'] = gpd.pd.to_datetime(gf.datetime)
gf.datetime.dtype

datetime64[ns, UTC]

In [94]:
# Change to datetime index, dropping timezone
gf['time'] = gpd.pd.to_datetime(gf.datetime) 

gf = gf.set_index('time').tz_localize(None) 

# Note you can also add new columns with whatever you want!
gf['stac_id'] = [item.id for item in items]

gf.head(3)

,geometry,datetime,platform,proj:epsg,instruments,s2:mgrs_tile,constellation,s2:granule_id,eo:cloud_cover,s2:datatake_id,...,s2:nodata_pixel_percentage,s2:unclassified_percentage,s2:dark_features_percentage,s2:not_vegetated_percentage,s2:degraded_msi_data_percentage,s2:high_proba_clouds_percentage,s2:reflectance_conversion_factor,s2:medium_proba_clouds_percentage,s2:saturated_defective_pixel_percentage,stac_id
time,,,,,,,,,,,,,,,,,,,,,
2023-03-29 13:47:09.024,"POLYGON ((-60.29520 -35.19821, -59.09015 -35.2...",2023-03-29 13:47:09.024000+00:00,Sentinel-2B,32721,[msi],21HTA,Sentinel 2,S2B_OPER_MSI_L2A_TL_MSFT_20230329T214329_A0316...,4.454349,GS2B_20230329T134709_031653_N05.09,...,0.000013,0.312079,0.000000,27.658752,0.0107,1.503456,1.005899,1.890754,0.0,S2B_MSIL2A_20230329T134709_R024_T21HTA_2023032...
2023-03-27 13:57:01.024,"POLYGON ((-59.25299 -32.51632, -59.28378 -32.6...",2023-03-27 13:57:01.024000+00:00,Sentinel-2A,32721,[msi],21HTD,Sentinel 2,S2A_OPER_MSI_L2A_TL_MSFT_20230327T232850_A0405...,0.005178,GS2A_20230327T135701_040533_N05.09,...,29.510382,1.511557,0.014379,21.916012,0.0307,0.000014,1.007036,0.000061,0.0,S2A_MSIL2A_20230327T135701_R067_T21HTD_2023032...
2023-03-27 13:57:01.024,"POLYGON ((-59.49005 -33.41271, -59.51840 -33.5...",2023-03-27 13:57:01.024000+00:00,Sentinel-2A,32721,[msi],21HTC,Sentinel 2,S2A_OPER_MSI_L2A_TL_MSFT_20230327T232831_A0405...,0.003199,GS2A_20230327T135701_040533_N05.09,...,47.621429,0.001013,0.000038,38.463461,0.0218,0.000000,1.007036,0.000025,0.0,S2A_MSIL2A_20230327T135701_R067_T21HTC_2023032...


In [95]:
odc.stac.stac_load?

In [96]:
%%time
ds = odc.stac.stac_load(
    items, 
    bands=["nir", "red"],
    bbox= area_of_interest,
    crs="utm",
    resolution= 250, # units of output CRS (UTM)
    groupby='solar_day', # adjacent acquisition frames automatically merged
    chunks={}, # Use Dask library to divy up large dataset!
)

ds

Wall time: 4.12 s


<xarray.Dataset>
Dimensions:      (y: 1054, x: 746, time: 192)
Coordinates:
  * y            (y) float64 6.182e+06 6.182e+06 ... 5.919e+06 5.919e+06
  * x            (x) float64 2.246e+05 2.249e+05 ... 4.106e+05 4.109e+05
    spatial_ref  int32 32721
  * time         (time) datetime64[ns] 2022-01-03T13:51:09.024000 ... 2023-03...
Data variables:
    nir          (time, y, x) float32 dask.array<chunksize=(1, 1054, 746), meta=np.ndarray>
    red          (time, y, x) float32 dask.array<chunksize=(1, 1054, 746), meta=np.ndarray>

In [97]:
print(ds)

<xarray.Dataset>
Dimensions:      (y: 1054, x: 746, time: 192)
Coordinates:
  * y            (y) float64 6.182e+06 6.182e+06 ... 5.919e+06 5.919e+06
  * x            (x) float64 2.246e+05 2.249e+05 ... 4.106e+05 4.109e+05
    spatial_ref  int32 32721
  * time         (time) datetime64[ns] 2022-01-03T13:51:09.024000 ... 2023-03...
Data variables:
    nir          (time, y, x) float32 dask.array<chunksize=(1, 1054, 746), meta=np.ndarray>
    red          (time, y, x) float32 dask.array<chunksize=(1, 1054, 746), meta=np.ndarray>


In [101]:
# Create a dataset with data that is different from zero
# Crear un conjunto de datos con los  datos que son diferentes de cero
ds = ds.where(ds!=0) 

# Calculate the NDVI | Caculamos el NDVI
ndvi = (ds.nir - ds.red) / (ds.nir + ds.red)

ndvi

<xarray.DataArray (time: 192, y: 1054, x: 746)>
dask.array<truediv, shape=(192, 1054, 746), dtype=float32, chunksize=(1, 1054, 746), chunktype=numpy.ndarray>
Coordinates:
  * y            (y) float64 6.182e+06 6.182e+06 ... 5.919e+06 5.919e+06
  * x            (x) float64 2.246e+05 2.249e+05 ... 4.106e+05 4.109e+05
    spatial_ref  int32 32721
  * time         (time) datetime64[ns] 2022-01-03T13:51:09.024000 ... 2023-03...

In [102]:
print('dataset size (MB):', ndvi.nbytes / 1e6)

dataset size (MB): 603.866112


In [103]:
%%time 

ndvi = ndvi.compute()

Aborting load due to failure while reading: https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/21/H/UB/2022/01/10/S2B_MSIL2A_20220110T134209_N0300_R124_T21HUB_20220111T213546.SAFE/GRANULE/L2A_T21HUB_A025318_20220110T134206/IMG_DATA/R10m/T21HUB_20220110T134209_B08_10m.tif?st=2023-04-16T15%3A27%3A18Z&se=2023-04-17T16%3A12%3A18Z&sp=rl&sv=2021-06-08&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2023-04-17T14%3A48%3A15Z&ske=2023-04-24T14%3A48%3A15Z&sks=b&skv=2021-06-08&sig=Ufo%2B0uSI8PM/IXA60xNpyMkH750cXdx/varHRBZ/zBM%3D:1
Aborting load due to failure while reading: https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/20/H/QG/2022/02/07/S2A_MSIL2A_20220207T135111_N0400_R024_T20HQG_20220220T024318.SAFE/GRANULE/L2A_T20HQG_A034627_20220207T135933/IMG_DATA/R10m/T20HQG_20220207T135111_B08_10m.tif?st=2023-04-16T15%3A27%3A18Z&se=2023-04-17T16%3A12%3A18Z&sp=rl&sv=2021-06-08&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86

RasterioIOError: '/vsicurl/https://sentinel2l2a01.blob.core.windows.net/sentinel2-l2/21/H/UB/2022/01/10/S2B_MSIL2A_20220110T134209_N0300_R124_T21HUB_20220111T213546.SAFE/GRANULE/L2A_T21HUB_A025318_20220110T134206/IMG_DATA/R10m/T21HUB_20220110T134209_B08_10m.tif?st=2023-04-16T15%3A27%3A18Z&se=2023-04-17T16%3A12%3A18Z&sp=rl&sv=2021-06-08&sr=c&skoid=c85c15d6-d1ae-42d4-af60-e2ca0f81359b&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2023-04-17T14%3A48%3A15Z&ske=2023-04-24T14%3A48%3A15Z&sks=b&skv=2021-06-08&sig=Ufo%2B0uSI8PM/IXA60xNpyMkH750cXdx/varHRBZ/zBM%3D' does not exist in the file system, and is not recognized as a supported dataset name.

In [ ]:
ndvi.isel(time=slice(0,5)).plot(col='time', vmin=-1, vmax=1, cmap='RdYlGn');

In [ ]:
pixel = ndvi.sel(x=5.9e5, y=5.18e6, method='nearest')

pixel

In [ ]:
monthly_means = ndvi.groupby('time').mean() 
monthly_means.plot(col='month', col_wrap=4, vmin=-1, vmax=1, cmap='RdYlGn')